In [2]:
import copy
import numpy as np
import os
import matplotlib.pyplot as plt
import pprint

from copy import deepcopy
from evo.core import metrics, sync
from evo.core.trajectory import PosePath3D
from evo.tools import plot

from tools.tum_tools import *
from tools.tum_plots import *
from tools.tum_definitions import *

%load_ext autoreload
%autoreload 2

%matplotlib inline
%matplotlib widget


In [ ]:
!evo_traj tum ekf_pose_with_covariance.tum --ref in_pose_with_covariance.tum -p

In [3]:
# Load previously computed data

data_root = "data/pickle/"

gt_file = os.path.join(data_root, "full", "gt_poses.pkl")
traj_file = os.path.join(data_root, "full2", "kiss_trajectories_corrected_corr.pkl")

gt_dict = read_pickle(gt_file)
traj_dict = read_pickle(traj_file)

In [6]:
# Extract relevant data
current_key = "mdv3_full_merge"
traj_ref = gt_dict['trajectory']
traj_est = traj_dict[current_key][2]
traj = (traj_ref, traj_est)


In [ ]:
plot_compare(traj, print_stats=False, plot_error='ape')

In [ ]:
plt.savefig(f"figures/ape_{current_key}.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# PLOT THE FULL SET
max_diff = 0.01
results = {}

for recording in Mdv3Dataset:
    if traj_dict.get(recording.value) is None:
        print(f"Info: Did not found a recorded trajectory for {recording.value}")
        continue
    else:
        print(f"Processing: {recording.value}")

    traj_ref = gt_dict['trajectory']
    traj_est = traj_dict[recording.value][2]
    ape_stats = plot_compare((traj_ref, traj_est), plot_error='ape', max_diff=max_diff)
    
    # plt.savefig(f"figures/ape_{recording.value}.pdf", format="pdf", bbox_inches="tight")
    results[recording.value] = ape_stats
    

In [ ]:
for recording, stats in results.items():
    print(f"Statistics for {recording}")
    pprint.pprint(stats)

In [ ]:
results

In [ ]:
import csv

fields = [ 'Type', 'max', 'mean', 'median', 'min', 'rmse', 'sse', 'std']


with open("test_output.csv", "w") as f:
    w = csv.DictWriter(f, fields)
    w.writeheader()
    for k in results:
        w.writerow({field: results[k].get(field) or k for field in fields})

In [ ]:
plot_trajectories_from_poses(traj_ref, traj_est)